# Natural Language Processing with Disaster Tweets
## Predict which Tweets are about real disasters and which ones are not

#### by Abhiram Rishi Prattipati

In [1]:
# Necessary libraries and tools

import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import xgboost
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train_dataframe = pd.read_csv("nlp_data/train.csv")

In [3]:
train_dataframe.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Tokenizes the string, removes strip handles, and replaces repeated characters
tweettoken = TweetTokenizer(strip_handles=True, reduce_len=True)
lemmatizer=WordNetLemmatizer()
stemmer=PorterStemmer()

In [5]:
collect=[]
def preprocess(t):
    # substitutes characters that are not alphaphetic to space
    tee=re.sub('[^a-zA-Z]'," ",t)
    # converts to lowercase
    tee=tee.lower()
    #tokenizes the tweet
    res=tweettoken.tokenize(tee)
    #removes stopwords
    for i in res:
        if i in stopwords.words('english'):
            res.remove(i)
    rest=[]
    # lemmatizes each word in res
    for k in res:
        rest.append(lemmatizer.lemmatize(k))
    # converts the list into string and spaces each element in list
    ret=" ".join(rest)
    # adds ret to collect
    collect.append(ret)

In [7]:
# preprocesses every tweet in the dataframe
for j in range(len(train_dataframe.index)):
    preprocess(train_dataframe["text"].iloc[j])

In [8]:
# converts a list of documents into a matrix array of total counts based on given vocabulary
def bow(ll):
    cv=CountVectorizer(max_features=200)
    x=cv.fit_transform(ll).toarray()
    return x
    

In [ ]:
y=bow(collect)